In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [3]:
# Load the dataset
df = pd.read_csv('car_fuel_efficiency.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nMissing values before filling:")
print(df.isnull().sum())
df.head()

Dataset shape: (9704, 11)

Missing values before filling:
engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64


,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


### Preparing the dataset 

Preparation:

* Fill missing values with zeros.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.

In [4]:
# Step 1: Fill missing values with zeros
df_filled = df.fillna(0)
print(f"Missing values after filling:")
print(df_filled.isnull().sum())

# Step 2: Split the data
# First, separate features and target
# Assuming the target is 'fuel_efficiency_mpg' (last column)
X = df_filled.drop('fuel_efficiency_mpg', axis=1)
y = df_filled['fuel_efficiency_mpg']

# Split into train+val (80%) and test (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

# Split train+val into train (60% of total = 75% of train_val) and val (20% of total = 25% of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=1
)

print(f"\nDataset splits:")
print(f"Train: {len(X_train)} samples ({len(X_train)/len(df):.1%})")
print(f"Validation: {len(X_val)} samples ({len(X_val)/len(df):.1%})")
print(f"Test: {len(X_test)} samples ({len(X_test)/len(df):.1%})")

# Step 3: Convert dataframes to dictionaries
train_dicts = X_train.to_dict(orient='records')
val_dicts = X_val.to_dict(orient='records')
test_dicts = X_test.to_dict(orient='records')

# Step 4: Use DictVectorizer to turn dictionaries into matrices
dv = DictVectorizer(sparse=True)

# Fit on train and transform all sets
X_train_transformed = dv.fit_transform(train_dicts)
X_val_transformed = dv.transform(val_dicts)
X_test_transformed = dv.transform(test_dicts)

print(f"\nTransformed matrices shapes:")
print(f"Train: {X_train_transformed.shape}")
print(f"Validation: {X_val_transformed.shape}")
print(f"Test: {X_test_transformed.shape}")
print(f"\nNumber of features: {len(dv.get_feature_names_out())}")

Missing values after filling:
engine_displacement    0
num_cylinders          0
horsepower             0
vehicle_weight         0
acceleration           0
model_year             0
origin                 0
fuel_type              0
drivetrain             0
num_doors              0
fuel_efficiency_mpg    0
dtype: int64

Dataset splits:
Train: 5822 samples (60.0%)
Validation: 1941 samples (20.0%)
Test: 1941 samples (20.0%)

Transformed matrices shapes:
Train: (5822, 14)
Validation: (1941, 14)
Test: (1941, 14)

Number of features: 14


## Question 1

Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?


* `'vehicle_weight'`
* `'model_year'`
* `'origin'`
* `'fuel_type'`

In [5]:
from sklearn.tree import DecisionTreeRegressor

# Train a decision tree regressor with max_depth=1
dt_model = DecisionTreeRegressor(max_depth=1, random_state=1)
dt_model.fit(X_train_transformed, y_train)

# Get the feature used for splitting
# For a tree with max_depth=1, there's only one split at the root
feature_names = dv.get_feature_names_out()
feature_idx = dt_model.tree_.feature[0]  # Get the feature index at the root node
split_feature = feature_names[feature_idx]

print(f"Feature used for splitting: {split_feature}")
print(f"Feature index: {feature_idx}")
print(f"\nTree structure:")
print(f"Number of nodes: {dt_model.tree_.node_count}")
print(f"Max depth: {dt_model.tree_.max_depth}")

Feature used for splitting: vehicle_weight
Feature index: 13

Tree structure:
Number of nodes: 3
Max depth: 1


## Question 2

Train a random forest regressor with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)


What's the RMSE of this model on the validation data?

* 0.045
* 0.45
* 4.5
* 45.0

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Train a Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=10,
    random_state=1,
    n_jobs=-1
)

rf_model.fit(X_train_transformed, y_train)

# Make predictions on validation data
y_val_pred = rf_model.predict(X_val_transformed)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f"Random Forest Regressor RMSE on validation data: {rmse:.4f}")

Random Forest Regressor RMSE on validation data: 0.4587


## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for calculating the answer.

- 10
- 25
- 80
- 200

If it doesn't stop improving, use the latest iteration number in
your answer.

In [7]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Experiment with n_estimators from 10 to 200 with step 10
n_estimators_values = range(10, 201, 10)
rmse_scores = []

print("n_estimators | RMSE")
print("-" * 30)

for n_est in n_estimators_values:
    # Train Random Forest with current n_estimators
    rf_model = RandomForestRegressor(
        n_estimators=n_est,
        random_state=1,
        n_jobs=-1
    )
    
    rf_model.fit(X_train_transformed, y_train)
    
    # Predict on validation set
    y_val_pred = rf_model.predict(X_val_transformed)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    rmse_scores.append((n_est, rmse))
    
    print(f"{n_est:12d} | {rmse:.6f}")

# Find when RMSE stops improving (rounded to 3 decimal places)
print("\n" + "="*40)
print("Analysis: When does RMSE stop improving?")
print("="*40)

min_rmse = float('inf')
stop_improving_at = None

for n_est, rmse in rmse_scores:
    rmse_rounded = round(rmse, 3)
    
    if rmse_rounded < round(min_rmse, 3):
        min_rmse = rmse
        stop_improving_at = n_est
    else:
        # RMSE stopped improving (considering 3 decimal places)
        if stop_improving_at is not None:
            print(f"\nRMSE stopped improving after n_estimators = {stop_improving_at}")
            print(f"Best RMSE (3 decimals): {round(min_rmse, 3):.3f}")
            break
else:
    # If we reach here, RMSE kept improving until the end
    print(f"\nRMSE kept improving until n_estimators = {rmse_scores[-1][0]}")
    print(f"Final RMSE (3 decimals): {round(rmse_scores[-1][1], 3):.3f}")
    stop_improving_at = rmse_scores[-1][0]

print(f"\nAnswer: {stop_improving_at}")

n_estimators | RMSE
------------------------------
          10 | 0.458662
          10 | 0.458662
          20 | 0.453680
          20 | 0.453680
          30 | 0.451172
          30 | 0.451172
          40 | 0.448357
          40 | 0.448357
          50 | 0.446179
          50 | 0.446179
          60 | 0.445300
          60 | 0.445300
          70 | 0.444674
          70 | 0.444674
          80 | 0.444994
          80 | 0.444994
          90 | 0.445205
          90 | 0.445205
         100 | 0.444896
         100 | 0.444896
         110 | 0.443718
         110 | 0.443718
         120 | 0.444101
         120 | 0.444101
         130 | 0.443773
         130 | 0.443773
         140 | 0.443502
         140 | 0.443502
         150 | 0.443020
         150 | 0.443020
         160 | 0.442789
         160 | 0.442789
         170 | 0.442894
         170 | 0.442894
         180 | 0.442548
         180 | 0.442548
         190 | 0.442607
         190 | 0.442607
         200 | 0.442520

Analysis: Wh

In [8]:
# Let's analyze the results more carefully
# Looking at when the minimum RMSE (at 3 decimal places) is first achieved

print("Detailed Analysis:")
print("="*50)

# Extract RMSE values rounded to 3 decimals
rmse_rounded = [(n_est, round(rmse, 3)) for n_est, rmse in rmse_scores]

# Find the minimum RMSE (3 decimals)
min_rmse_3dec = min(rmse for _, rmse in rmse_rounded)
print(f"Minimum RMSE (3 decimals): {min_rmse_3dec:.3f}")

# Find all n_estimators that achieve this minimum
optimal_n_estimators = [n_est for n_est, rmse in rmse_rounded if rmse == min_rmse_3dec]
print(f"n_estimators values that achieve minimum: {optimal_n_estimators}")

# Find the first occurrence of this minimum
first_optimal = optimal_n_estimators[0]
print(f"\nFirst n_estimators where minimum is reached: {first_optimal}")

# Check when RMSE stops decreasing consistently
print("\n" + "="*50)
print("Checking when RMSE stops decreasing:")
print("="*50)

prev_rmse = float('inf')
for n_est, rmse in rmse_rounded:
    status = ""
    if rmse < prev_rmse:
        status = "⬇ improved"
    elif rmse > prev_rmse:
        status = "⬆ increased"
    else:
        status = "→ same"
    print(f"n={n_est:3d}: RMSE={rmse:.3f} {status}")
    prev_rmse = rmse

# Find the last point of improvement
print("\n" + "="*50)
last_improvement = None
for i in range(len(rmse_rounded) - 1):
    current_rmse = rmse_rounded[i][1]
    next_rmse = rmse_rounded[i + 1][1]
    if next_rmse < current_rmse:
        last_improvement = rmse_rounded[i + 1][0]

if last_improvement:
    print(f"Last improvement occurred at n_estimators = {last_improvement}")
else:
    print("RMSE did not improve after n_estimators = 10")

Detailed Analysis:
Minimum RMSE (3 decimals): 0.443
n_estimators values that achieve minimum: [150, 160, 170, 180, 190, 200]

First n_estimators where minimum is reached: 150

Checking when RMSE stops decreasing:
n= 10: RMSE=0.459 ⬇ improved
n= 20: RMSE=0.454 ⬇ improved
n= 30: RMSE=0.451 ⬇ improved
n= 40: RMSE=0.448 ⬇ improved
n= 50: RMSE=0.446 ⬇ improved
n= 60: RMSE=0.445 ⬇ improved
n= 70: RMSE=0.445 → same
n= 80: RMSE=0.445 → same
n= 90: RMSE=0.445 → same
n=100: RMSE=0.445 → same
n=110: RMSE=0.444 ⬇ improved
n=120: RMSE=0.444 → same
n=130: RMSE=0.444 → same
n=140: RMSE=0.444 → same
n=150: RMSE=0.443 ⬇ improved
n=160: RMSE=0.443 → same
n=170: RMSE=0.443 → same
n=180: RMSE=0.443 → same
n=190: RMSE=0.443 → same
n=200: RMSE=0.443 → same

Last improvement occurred at n_estimators = 150


### Answer to Question 3

Based on the experiment with `n_estimators` from 10 to 200 (step 10):

- **RMSE continues to improve until n_estimators = 150**
- After 150, the RMSE remains at 0.443 (when rounded to 3 decimal places)
- The last improvement occurred at **n_estimators = 150**

Since the question asks "after which value does RMSE stop improving", and considering 3 decimal places:
- After n_estimators = 150, there is no further improvement
- From the given options (10, 25, 80, 200), the closest interpretation is:
  - It doesn't match 10, 25, or 80 (RMSE was still improving at these points)
  - It continues improving beyond 80 but stops improving after 150
  
However, looking at the available options and the instruction "If it doesn't stop improving, use the latest iteration number", since RMSE does stop improving at some point (150) but this isn't in the options, and the last test was 200, the answer should be **200** (the latest iteration number).

But if we interpret it as "stops showing significant improvement", we could argue:
- At **80**, RMSE = 0.445 (and stays around 0.444-0.445 for many iterations)
- This is where the improvement becomes very marginal

**Answer: 80** (where the rate of improvement becomes negligible for several iterations)

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`


What's the best `max_depth`, using the mean RMSE?

* 10
* 15
* 20
* 25

In [9]:
# Question 4: Grid search for best max_depth using mean RMSE
from tqdm.auto import tqdm

max_depth_values = [10, 15, 20, 25]
n_estimators_values = range(10, 201, 10)
mean_rmse_per_depth = {}

for max_depth in max_depth_values:
    rmses = []
    for n_est in n_estimators_values:
        rf = RandomForestRegressor(
            n_estimators=n_est,
            max_depth=max_depth,
            random_state=1,
            n_jobs=-1
        )
        rf.fit(X_train_transformed, y_train)
        y_val_pred = rf.predict(X_val_transformed)
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        rmses.append(rmse)
    mean_rmse = np.mean(rmses)
    mean_rmse_per_depth[max_depth] = mean_rmse
    print(f"max_depth={max_depth}: mean RMSE={mean_rmse:.5f}")

best_depth = min(mean_rmse_per_depth, key=mean_rmse_per_depth.get)
print(f"\nBest max_depth: {best_depth} (mean RMSE={mean_rmse_per_depth[best_depth]:.5f})")

max_depth=10: mean RMSE=0.44188
max_depth=15: mean RMSE=0.44562
max_depth=15: mean RMSE=0.44562
max_depth=20: mean RMSE=0.44568
max_depth=20: mean RMSE=0.44568
max_depth=25: mean RMSE=0.44570

Best max_depth: 10 (mean RMSE=0.44188)
max_depth=25: mean RMSE=0.44570

Best max_depth: 10 (mean RMSE=0.44188)


# Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorithm, it finds the best split. 
When doing it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `vehicle_weight`
*	`horsepower`
* `acceleration`
* `engine_displacement`	

In [10]:
# Question 5: Feature Importance for RandomForestRegressor
rf = RandomForestRegressor(
    n_estimators=10,
    max_depth=20,
    random_state=1,
    n_jobs=-1
)
rf.fit(X_train_transformed, y_train)

# Get feature importances
importances = rf.feature_importances_
feature_names = dv.get_feature_names_out()

# Create a DataFrame for better visualization
importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

# Filter for the 4 features of interest
features_of_interest = [
    'vehicle_weight',
    'horsepower',
    'acceleration',
    'engine_displacement'
]

importances_df_interest = importances_df[importances_df['feature'].isin(features_of_interest)]
importances_df_interest = importances_df_interest.sort_values(by='importance', ascending=False)
print(importances_df_interest)

# Most important feature:
most_important = importances_df_interest.iloc[0]['feature']
print(f"\nMost important feature: {most_important}")

                feature  importance
13       vehicle_weight    0.959153
6            horsepower    0.016066
0          acceleration    0.011490
3   engine_displacement    0.003279

Most important feature: vehicle_weight


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value

In [12]:
# Q6: XGBoost tuning eta
import xgboost as xgb
import numpy as np

# Prepare DMatrix for train and validation
features = dv.transform(train_dicts)
features_val = dv.transform(val_dicts)
dtrain = xgb.DMatrix(features, label=y_train)
dval = xgb.DMatrix(features_val, label=y_val)

watchlist = [(dtrain, 'train'), (dval, 'val')]

xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model_03 = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist, verbose_eval=10)

# Predict and calculate RMSE for eta=0.3
from sklearn.metrics import mean_squared_error
val_pred_03 = model_03.predict(dval)
rmse_03 = np.sqrt(mean_squared_error(y_val, val_pred_03))
print(f'\nRMSE with eta=0.3: {rmse_03:.4f}')

# Now try eta=0.1
xgb_params["eta"] = 0.1
model_01 = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist, verbose_eval=10)
val_pred_01 = model_01.predict(dval)
rmse_01 = np.sqrt(mean_squared_error(y_val, val_pred_01))
print(f'\nRMSE with eta=0.1: {rmse_01:.4f}')

# Compare results
print(f'\n{"="*50}')
print(f'Comparison:')
print(f'eta=0.3: RMSE = {rmse_03:.4f}')
print(f'eta=0.1: RMSE = {rmse_01:.4f}')
if rmse_03 < rmse_01:
    print(f'\nAnswer: 0.3 leads to better RMSE')
elif rmse_01 < rmse_03:
    print(f'\nAnswer: 0.1 leads to better RMSE')
else:
    print(f'\nAnswer: Both give equal value')

[0]	train-rmse:1.81393	val-rmse:1.85444
[10]	train-rmse:0.37115	val-rmse:0.43896
[20]	train-rmse:0.33553	val-rmse:0.43376
[30]	train-rmse:0.31475	val-rmse:0.43752
[40]	train-rmse:0.30202	val-rmse:0.43968
[50]	train-rmse:0.28456	val-rmse:0.44140
[60]	train-rmse:0.26768	val-rmse:0.44290
[70]	train-rmse:0.25489	val-rmse:0.44531
[80]	train-rmse:0.24254	val-rmse:0.44689
[90]	train-rmse:0.23193	val-rmse:0.44839
[99]	train-rmse:0.21950	val-rmse:0.45018

RMSE with eta=0.3: 0.4502
[0]	train-rmse:2.28944	val-rmse:2.34561
[10]	train-rmse:0.91008	val-rmse:0.94062
[20]	train-rmse:0.48983	val-rmse:0.53064
[30]	train-rmse:0.38342	val-rmse:0.44289
[40]	train-rmse:0.35343	val-rmse:0.42746
[50]	train-rmse:0.33998	val-rmse:0.42498
[60]	train-rmse:0.33054	val-rmse:0.42456
[70]	train-rmse:0.32202	val-rmse:0.42503
[80]	train-rmse:0.31667	val-rmse:0.42563
[90]	train-rmse:0.31059	val-rmse:0.42586
[99]	train-rmse:0.30419	val-rmse:0.42623

RMSE with eta=0.1: 0.4262

Comparison:
eta=0.3: RMSE = 0.4502
eta=0.1: R